In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
RATING_PATH = '../data/raw/Number_of_ratings.xlsx'
DEFAULT_PATH = '../data/raw/Number_of_defaults.xlsx'

ratings = pd.read_excel(RATING_PATH).T
defaults = pd.read_excel(DEFAULT_PATH).T

ratings = ratings.iloc[1:, [7, 6, 5, 4, 3]]
ratings.to_csv("../data/processed/ratings.csv", index=False, header=False)

defaults = defaults.iloc[1:, [7, 6, 5, 4, 3]]
defaults.to_csv("../data/processed/defaults.csv", index=False, header=False)
defaults

,7,6,5,4,3
Number of defaults,41,22,3,0,0
Unnamed: 2,37,26,1,0,0
Unnamed: 3,31,14,0,0,0
Unnamed: 4,66,71,6,0,0
Unnamed: 5,53,19,0,0,0
Unnamed: 6,61,31,0,2,0
Unnamed: 7,113,73,12,0,0
Unnamed: 8,50,10,0,0,0
Unnamed: 9,44,23,4,0,0
Unnamed: 10,101,24,2,2,0
